# TP4 : Techniques avancées

Dans ce TP, vous allez mettre en pratique tout ce que vous avez appris !

Ce TP sera **À RENDRE** sur Moodle :
- Fichier notebook (`tp4.ipynb`) complété avec vos réponses
- Fichier `models_results.csv` détaillant les résultats de vos expérimentations (voir Exercice 1, Q5 et Q6)
- Fichier `predictions.csv` contenant les prédictions de votre meilleur modèle sur les données d'utilisation (voir Exercice 1, Q7)

Il comptera pour **5 points** dans votre note de TP de l'UE, principalement déterminé en fonction du F2-score de vos prédictions sur les données d'utilisation (dont vous n'aurez pas les labels...).
Exemple : vous obtenez un F2-score de 0.68 sur les données d'utilisation, vous obtiendrez $0.68 * 5 = 3.4$ points.

Le prof reste souverain et s'autorise à baisser ou invalider la note si votre fichier notebook (`tp4.ipynb`) contient de la triche et/ou ne correspond pas aux résultats que vous fournissez ! Soyez donc honnête :-)

Dans la limite de l'utilisation des données **de "prototypage"** fournies, vous êtes autorisés à utiliser toutes les techniques vues en cours : encodages, normalisations, équilibrages, suppression ou remplacement des données manquantes, ...

Les données dites "de déploiement" (ou d'utilisation) doivent être considérées comme inaccessibles : vous ne devrez les utiliser que pour faire vos prédictions finales. Comme si vous les receviez une fois votre modèle de ML entraîné et déployé sur un serveur.

Dans ce TP, on se place dans la situation de *data scientists* dans un hôpital. Vos collègues médecins essaient de détecter la présence d'une maladie cardiaque à partir de données sur les patients, obtenues par divers examens médicaux.

Votre responsable hiérarchique veut que vous entraîniez un modèle de ML qui sera capable de prédire la présence de cette maladie, afin d'accélérer le traitement des patients et d'éviter le recours à des examens complémentaires beaucoup plus coûteux.

On veut donc identifier le plus possible de patients malades (= maximiser le rappel !) pour laisser mourir le moins de patients possibles, tout en évitant de prédire les patients sains comme malade (= éviter que la précision ne soit de 0 !), sinon cela prendra trop de temps et coûtera trop cher... Nous utiliserons donc un F2-score comme métrique principale de sélection du meilleur modèle. Les autres métriques seront calculées et mémorisées à titre indicatif.

Description des données :

| Nom | Description |
|-----|-------------|
| age | Âge du patient en années |
| sex | Sexe du patient (`male` / `female`) |
| chest pain | Type de douleur dans la poitrine ressentie |
| resting blood pressure (mm Hg) | Pression sanguine au repos |
| cholestoral (mg/dl) | Taux de choléstérol dans le sang |
| fasting blood sugar > 120 mg/dl | Est-ce que le taux de glucose dans le sang après 8h à jeûn dépasse une valeur normale ? |
| resting electrocardiograph | Présence d'une anomalie à l'électrocardiographe (ECG) au repos (`normal` : pas d'anomalie, `abnormal` : anomalie ST-T, `hypertrophy` : probable hypertrophie du ventricule gauche) |
| max heart rate | Nombre maximum de battements cardiaques par minutes durant un exercice de résistance à l'effort (ex : vélo) |
| exercise induced angina | Est-ce que l'exercice de résistance à l'effort a entraîné une douleur à la poitrine ? |
| oldpeak | Dépression du segment ST de l'ECG lors de l'exercice de résistance à l'effort (relativement à l'ECG au repos) |
| slope | Forme du segment ST de l'ECG (`upsloping` : hausse, `flat` : plat, `downsloping` : baisse) |
| number of colored vessels by fluoroscopy | Le nombre d'artères majeures colorées par fluoroscopie (rayons X ou CT-scan) |
| thalassemia | Présence d'une maladie sanguine (`3` : normal, `6` : maladie irréversible, `7` : maladie réversible)
| **class** | **Cible de prédiction** : présence (`disease`) ou absence (`no disease`) d'un problème cardiaque |

In [318]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

## Exercice 1 - Prise en main des données et préparation des fichiers de réponses

J'attends un certain format dans vos fichiers de réponses (`models_results.csv` et `predictions.csv`) ; cet exercice va vous faire implémenter les fonctions permettant de remplir ces fichiers comme attendu.

Comme vu dans le CM4, lorsque l'on prototype et expérimente sur divers algorithmes de ML, il est important de retenir les scores obtenus pour chaque algorithme, chaque préparation de données, et chaque *seed* (graine aléatoire). Cela permettra de sélectionner le meilleur modèle une fois les expérimentations effectuées.

Q1. Chargez le fichier `data_prototyping.csv` dans un *DataFrame* `df`.

In [319]:
# Charger les fichiers CSV en séparant les données par des points-virgules
df = pd.read_csv('data_prototyping.csv', na_values='?')

# Afficher un aperçu des données
print(df.head())

   age     sex        chest pain  resting blood pressure (mm Hg)  \
0   44    male      asymptomatic                           120.0   
1   36    male  non-anginal pain                           130.0   
2   39  female  non-anginal pain                           110.0   
3   48    male      asymptomatic                           124.0   
4   36    male  non-anginal pain                           150.0   

   cholestoral (mg/dl) fasting blood sugar > 120 mg/dl  \
0                169.0                              no   
1                209.0                              no   
2                182.0                              no   
3                274.0                              no   
4                160.0                              no   

  resting electrocardiograph  max heart rate exercise induced angina  oldpeak  \
0                     normal           144.0                     yes      2.8   
1                     normal           178.0                      no      0.0   

Q2. Faites une EDA minimale : quelles sont les colonnes, quels sont leurs types de données ? Quelles sont les valeurs uniques pour chaque colonne catégorielle ? Existe-t-il des valeurs manquantes ?

Vous ferez une EDA plus poussée dans l'Exercice 2.

In [320]:
print("Columns dans le DataFrame:")
print(df.columns.tolist())

Columns dans le DataFrame:
['age', 'sex', 'chest pain', 'resting blood pressure (mm Hg)', 'cholestoral (mg/dl)', 'fasting blood sugar > 120 mg/dl', 'resting electrocardiograph', 'max heart rate', 'exercise induced angina', 'oldpeak', 'slope', 'number of colored vessels by fluoroscopy', 'thalassemia', 'class']


In [321]:
# Identifier les colonnes catégorielles
categorical_columns = df.select_dtypes(include=['object']).columns

# Afficher les valeurs uniques pour chaque colonne catégorielle
for col in categorical_columns:
    print(f"Valeurs uniques pour la colonne {col}: {df[col].unique()}")


Valeurs uniques pour la colonne sex: ['male' 'female']
Valeurs uniques pour la colonne chest pain: ['asymptomatic' 'non-anginal pain' 'atypical angina' 'typical angina']
Valeurs uniques pour la colonne fasting blood sugar > 120 mg/dl: ['no' 'yes' nan]
Valeurs uniques pour la colonne resting electrocardiograph: ['normal' 'abnormal' 'hypertrophy']
Valeurs uniques pour la colonne exercise induced angina: ['yes' 'no' nan]
Valeurs uniques pour la colonne slope: ['downsloping' nan 'flat' 'upsloping']
Valeurs uniques pour la colonne class: ['disease' 'no disease']


In [322]:
# Vérifier la présence de valeurs manquantes
missing_values = df.isnull().sum()

# Afficher les colonnes avec des valeurs manquantes
print(missing_values[missing_values > 0])


resting blood pressure (mm Hg)               25
cholestoral (mg/dl)                          22
fasting blood sugar > 120 mg/dl              23
max heart rate                               25
exercise induced angina                      25
oldpeak                                      27
slope                                       198
number of colored vessels by fluoroscopy    284
thalassemia                                 253
dtype: int64


Q3. Séparez vos données en `df_train` + `df_test`. Vous êtes libre de choisir la proportion de données dans chaque jeu.

In [323]:
from sklearn.model_selection import train_test_split

# Séparer les données en jeu d'entraînement et jeu de test (80% entraînement / 20% test)
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)

# Vérifier les dimensions des deux DataFrames
print(f"Dimensions de df_train: {df_train.shape}")
print(f"Dimensions de df_test: {df_test.shape}")

Dimensions de df_train: (367, 14)
Dimensions de df_test: (92, 14)


Q3. On veut préparer les données de manière à ce qu'elles soient utilisables par un algorithme de ML. Il existe plusieurs façons de les préparer, comme on l'a vu au TP2 (c'est subjectif !). Chaque façon peut donner des performances différentes ; certains algorithmes fonctionneront mieux avec certaines façons que d'autres...

Implémentez la fonction `prepare_data_v1(df)` qui prépare les données d'un *DataFrame* en entrée, de manière la plus simple possible : je recommande un **encodage ordinal** pour commencer et pouvoir tester rapidement vos algorithmes. Supprimez les colonnes contenant trop de NAs (>50%) pour simplifier également, ainsi que les lignes contenant au moins 1 NA.

**Attention** : on rappelle que cet encodage fait perdre toute capacité d'interprétation s'il est utilisé sur une variable qui n'a pas de relation d'ordre ! Certains algorithmes auront également de mauvaises performances car ils calculeront une relation statistique qui n'existe pas... Ce n'est pas grave pour cet exercice, cela permet de tester rapidement un algorithme sur nos données.

**Attention²** : Votre fonction devra fonctionner sur des sous-ensembles de données ! En effet, en pratique vous ne connaîtrez pas vos données d'utilisation avant d'avoir déployé votre modèle... Votre fonction de préparation devra donc être appliquée sur les données d'utilisations indépendamment des données de prototypage. Faites particulièrement attention à **l'ordre de vos encodages** ! Si votre fonction encode `normal=1, abnormal=2` dans vos données de prototypage, puis `abnormal=1, normal=2` dans vos données d'utilisation, votre modèle de ML prédira n'importe quoi... Je recommande de faire votre encodage à la main, en utilisant par exemple [`df.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) pour vous assurer que l'encodage sera **cohérent**.

In [324]:
def prepare_data_v1(df):
    # Remplacer les '?' par NaN
    df.replace('?', np.nan, inplace=True)
    
    # Imputer les valeurs manquantes
    for col in df.columns:
        if df[col].dtype == 'object':
            # Imputation pour les colonnes catégorielles
            df[col].fillna(df[col].mode()[0], inplace=True)  # Valeur la plus fréquente
        else:
            # Imputation pour les colonnes numériques
            df[col].fillna(df[col].median(), inplace=True)  # Médiane

    # Supprimer les colonnes avec plus de 50% de valeurs manquantes
    threshold = len(df) * 0.5
    df = df.dropna(axis=1, thresh=threshold)

    # Encodage manuel des colonnes catégorielles (sans encoder la cible)
    if 'sex' in df.columns:
        df['sex'] = df['sex'].replace({'male': 1, 'female': 0})
        
    if 'chest pain' in df.columns:
        df['chest pain'] = df['chest pain'].replace({'typical angina': 1, 'atypical angina': 2, 'non-anginal pain': 3, 'asymptomatic': 4})
    
    if 'resting electrocardiograph' in df.columns:
        df['resting electrocardiograph'] = df['resting electrocardiograph'].replace({'normal': 0, 'abnormal': 1, 'hypertrophy': 2})
    
    if 'fasting blood sugar > 120 mg/dl' in df.columns:
        df['fasting blood sugar > 120 mg/dl'] = df['fasting blood sugar > 120 mg/dl'].replace({'no': 0, 'yes': 1})
    
    if 'exercise induced angina' in df.columns:
        df['exercise induced angina'] = df['exercise induced angina'].replace({'no': 0, 'yes': 1})
    
    if 'slope' in df.columns:
        df['slope'] = df['slope'].replace({'upsloping': 1, 'flat': 2, 'downsloping': 3})
    
    if 'thalassemia' in df.columns:
        df['thalassemia'] = df['thalassemia'].replace({'3': 0, '6': 1, '7': 2})

    # Conversion des colonnes numériques
    for col in ['resting blood pressure (mm Hg)', 'cholestoral (mg/dl)', 'max heart rate', 'oldpeak', 'number of colored vessels by fluoroscopy']:
        if col in df.columns:
            df[col] = df[col].astype(float)

    # S'assurer que 'thalassemia' est toujours présente
    if 'thalassemia' not in df.columns:
        df['thalassemia'] = 3  # Valeur par défaut

    return df

Si vous avez bien séparés vos données et implémenté la fonction de préparation, la prochaine cellule devrait s'exécuter sans problème. Notez bien que les données de test ne sont pas préparées en même temps que les données d'entraînement !

In [325]:
df_train_v1, df_test_v1 = prepare_data_v1(df_train), prepare_data_v1(df_test)

/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3761464116.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)  # Médiane
/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3761464116.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

In [326]:
# Afficher un aperçu des données préparées
print(df_train_v1.head())
print(df_test_v1.head())

     age  sex  chest pain  resting blood pressure (mm Hg)  \
109   51    0           3                           150.0   
270   55    1           3                           110.0   
194   49    1           4                           140.0   
357   32    1           2                           125.0   
164   58    1           3                           105.0   

     cholestoral (mg/dl)  fasting blood sugar > 120 mg/dl  \
109                200.0                                0   
270                277.0                                0   
194                228.0                                0   
357                254.0                                0   
164                240.0                                0   

     resting electrocardiograph  max heart rate  exercise induced angina  \
109                           0           120.0                        0   
270                           0           160.0                        0   
194                           0       

Q5. Implémentez la méthode `evaluate_model(df_train, df_test, data_name, model_name, model_class, hyperparameters)`. Cette méthode devra instancier un estimateur (`model_class`) avec les hyperparamètres demandés, l'entraîner sur les données d'entraînement fournies, puis calculer les métriques suivantes sur les données de test fournies :

- F2-score
- Accuracy
- Precision
- Recall

Vous pouvez utiliser les méthodes de Scikit-learn pour ces [métriques](https://scikit-learn.org/stable/api/sklearn.metrics.html) (attention à la classe d'intérêt ! Par défaut, Scikit utilise `1` comme classe d'intérêt, mais vous pouvez en spécifier une autre via l'argument `pos_label`).

Votre fonction devra renvoyer un *DataFrame* contenant les colonnes suivantes :
- ModelName : le nom du modèle utilisé (algorithme)
- Data : le nom de la méthode de préparation des données
- Hyperparamètres : le dictionnaire des hyperparamètres
- RandomSeed : la graine aléatoire utilisée pour instancier et entraîner le modèle
- Score_f2 : la métrique de F2-Score sur les prédictions de test
- Score_precision : la métrique de Precision sur les prédictions de test
- Score_recall : la métrique de Rappel sur les prédictions de test
- Score_accuracy : la métrique d'Accuracy (% de prédictions correctes) sur les prédictions de test
- TrainedEstimator : l'instance du modèle entraînée (pour réutilisation ultérieure si besoin)


On vous donne le squelette de cette fonction, à vous de remplir les trous !

```python
def evaluate_model(df_train, df_test, data_name, model_name, model_class, hyperparameters):
    X_train = # À COMPLÉTER
    Y_train = # À COMPLÉTER
    X_test = # À COMPLÉTER
    Y_test = # À COMPLÉTER

    # On génère une graine aléatoire ... de manière aléatoire :-)
    seed = np.random.randint(0, 2**32 - 1)
    np.random.seed(seed)

    # On instancie l'algorithme avec les hyperparamètres demandés
    # En pratique, si on a par exemple `model_class=LogisticRegression` et
    # # `hyperparameters={ 'n_iter': 100, 'eta0': 0.1 }`, c'est équivalent à faire
    # `LogisticRegression(n_iter=100, eta0=0.1)`
    estimator = model_class(**hyperparameters)

    # À COMPLÉTER : entraînez le modèle !
    
    y_pred = # À COMPLÉTER : calculez les prédictions sur les données de test

    return pd.DataFrame([{
        'ModelName': model_name,
        'Data': data_name,
        'Hyperparameters': hyperparameters,
        'RandomSeed': seed,
        'Score_f2': # À COMPLÉTER
        'Score_precision': # À COMPLÉTER
        'Score_recall': # À COMPLÉTER
        'Score_accuracy': # À COMPLÉTER
        'TrainedEstimator': estimator
    }])
```

In [327]:
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score

def evaluate_model(df_train, df_test, data_name, model_name, model_class, hyperparameters):
    # Vérifier si la colonne 'class' est présente dans df_train et df_test
    if 'class' not in df_train.columns or 'class' not in df_test.columns:
        raise KeyError("'class' column not found in df_train or df_test")
    
    # Séparation des features (X) et de la cible (Y)
    X_train = df_train.drop(columns=['class'])
    Y_train = df_train['class']
    X_test = df_test.drop(columns=['class'])
    Y_test = df_test['class']
    
    # Générer une graine aléatoire
    seed = np.random.randint(0, 2**32 - 1)
    np.random.seed(seed)
    
    # Instancier le modèle avec les hyperparamètres
    estimator = model_class(**hyperparameters)
    
    # Entraîner le modèle
    estimator.fit(X_train, Y_train)
    
    # Prédictions sur les données de test
    y_pred = estimator.predict(X_test)
    
    # Calcul des métriques avec pos_label='disease'
    score_f2 = fbeta_score(Y_test, y_pred, beta=2, pos_label='disease')
    score_precision = precision_score(Y_test, y_pred, pos_label='disease')
    score_recall = recall_score(Y_test, y_pred, pos_label='disease')
    score_accuracy = accuracy_score(Y_test, y_pred)
    
    # Retourner les résultats
    return pd.DataFrame([{
        'ModelName': model_name,
        'Data': data_name,
        'Hyperparameters': hyperparameters,
        'RandomSeed': seed,
        'Score_f2': score_f2,
        'Score_precision': score_precision,
        'Score_recall': score_recall,
        'Score_accuracy': score_accuracy,
        'TrainedEstimator': estimator
    }])

Si vous avez bien implémenté la fonction `evaluate_model()` et que vos données sont bien préparées, l'expérimentation ci-dessous devrait fonctionner et vous afficher le résultat d'une Régression Logistique (sur des données préparées "naïvement").

In [328]:
from sklearn.linear_model import LogisticRegression

# Entraîner et évaluer le modèle
results = evaluate_model(
    df_train_v1,
    df_test_v1,
    'v1',
    'LogisticRegression',
    LogisticRegression,
    {'penalty': None, 'max_iter': 1000}
)

results

,ModelName,Data,Hyperparameters,RandomSeed,Score_f2,Score_precision,Score_recall,Score_accuracy,TrainedEstimator
0,LogisticRegression,v1,"{'penalty': None, 'max_iter': 1000}",657861340,0.45977,0.533333,0.444444,0.815217,"LogisticRegression(max_iter=1000, penalty=None)"


Q6. Enregistrez les résultats de l'expérimentation ci-dessus dans un fichier `models_results.csv`.

On vous donne, pour vous aider, une fonction qui ajoute automatiquement les résultats sans effacer les précédents ! :-)

Note : il est très important d'enregistrer vos résultats à chaque expérimentation, pour ne pas "perdre" ces résultats, en particulier si vous redémarrez votre notebook. Dans la suite du TP, utilisez cette fonction **chaque fois** que vous faites une nouvelle expérimentation ! En pratique, on utilise souvent des outils dédiés tels que ceux présentés en cours (MLFlow, Weights and Biases, Aimstack, ...).

Le fichier `models_results.csv` sera à rendre.

In [329]:
def append_results(results):
    with open('models_results.csv', 'a') as f:
        # Quand le fichier est vide, il faut préciser les colonnes ;
        # si on a au moins une ligne, il ne faut pas répéter les noms des colonnes...
        header = f.seek(0, 2) == 0
        # On ne veut pas écrire la colonne `TrainedEstimator` (c'est un objet Python !)
        columns = [
            'ModelName', 'Data', 'Hyperparameters', 'RandomSeed', 'Score_f2', 
            'Score_precision', 'Score_recall', 'Score_accuracy',
        ]
        results.to_csv(f, header=header, columns=columns, index=False)

In [330]:
# Appeler la fonction pour ajouter les résultats dans le fichier CSV
append_results(results)

# Vérification : Afficher les résultats enregistrés
print("Les résultats ont été enregistrés dans 'models_results.csv'.")


Les résultats ont été enregistrés dans 'models_results.csv'.


Q7. Pour vous montrer ce qu'on attend pour les prédictions à rendre, faites les prédictions sur le modèle de Régression Logistique de la précédente expérimentation, sur les données "de déploiement", préparées via la façon 'v1'.

On vous fournit une fonction pour écrire les prédictions dans un fichier `predictions.csv`.

**Attention** : dans la suite du TP, vous ne devrez faire les prédictions sur ces données de déploiement que pour votre meilleur modèle et votre meilleure préparation de données ! De toute façon vous n'aurez pas les labels, donc vous ne saurez pas si vos prédictions sont bonnes ou non...

1. Récupérez le modèle entraîné (`TrainedEstimator`) de votre meilleure expérimentation (pour l'instant vous n'en avez qu'une seule : `results`). Si vous redémarrez le notebook d'ici la fin du TP, vous devriez avoir toutes les informations dans le fichier `models_results.csv` (la classe à utiliser, la fonction de préparation de données, les hyperparamètres, et la *seed* aléatoire) pour ré-entraîner le même modèle.
2. Puis, appelez la fonction `write_predictions()` avec en paramètre votre fonction de préparation de données (ici, `prepare_data_v1`) et votre modèle entraîné.

Cette fonction se charge automatiquement de lire les données, les préparer, faire les prédictions, et les écrire dans le fichier de sortie `predictions.csv`. Elle gère également les prédictions manquantes si votre fonction de préparations de données supprime des lignes (NAs).

Le fichier `predictions.csv` sera à rendre.

In [331]:
def write_predictions(prepare_data_function, estimator):
    df_deployment = pd.read_csv('data_deployment.csv')
    df_deployment_prepared = prepare_data_function(df_deployment)
    # /!\ Si la fonction de préparation des données supprime des lignes
    # (par exemple à cause de la gestion des NAs), on va se retrouver 
    # avec moins de prédictions qu'on a de lignes...
    # On doit matcher chaque prédiction avec sa ligne correspondante
    handled_lines = df_deployment_prepared.index

    # On prédit pour chaque ligne des données préparées
    predictions = estimator.predict(df_deployment_prepared)
    # On ré-assimile aux numéros de lignes des données originales
    predictions = pd.Series(predictions, index=handled_lines)

    # On ajoute la colonne aux données originales ; puisqu'on a indexé les
    # prédictions, les prédictions manquantes seront automatiquement remplacées par NaN
    df_deployment['predictions'] = predictions
    # On écrit le fichier
    df_deployment.to_csv('predictions.csv', index=True)

In [332]:
trained_estimator = results['TrainedEstimator'].iloc[0]  # Récupérer le modèle entraîné

write_predictions(prepare_data_v1, trained_estimator)    # Utiliser prepare_data_v1 pour préparer les données de prédiction

# Vérification
print("Les prédictions ont été enregistrées dans 'predictions.csv'.")

Les prédictions ont été enregistrées dans 'predictions.csv'.


/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3761464116.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)  # Médiane
/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3761464116.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

## Exercice 2 : à vous de jouer !

Vous êtes maintenant libres de vos choix pour identifier le meilleur modèle et la meilleure préparation de données !

N'oubliez pas d'enregistrer chaque expérimentation via la méthode `append_results()`.

**Attention** : une fois que vous avez expérimenté avec une façon de préparer de vos données (par exemple, `prepare_data_v1()`), vous NE DEVEZ PLUS modifier cette fonction ! Sinon, vous rendez irreproductibles toutes vos expérimentations précédentes ! Dans la pratique, on utilise souvent des Data Version Control (DVC) et des Version Control System (VCS, exemple : Git) pour s'assurer de mémoriser ce qu'on a fait et de pouvoir y revenir.

Vous devrez donc implémenter des fonctions `prepare_data_v2()`, `prepare_data_v3()`, etc. (autant que nécessaires pour tester toutes vos idées) !

On vous donne quelques pistes d'exploration ci-dessous pour vous aider ...
Les questions suivantes sont données à titre d'indication, vous n'êtes pas obligés de toutes les faire, ni de suivre cet ordre.

Q1. Faites une EDA plus poussée. Quelles sont les données catégorielles ? Quelles valeurs peuvent-elles prendre ? Existe-t-il des données manquantes ?

Astuce : les types de données que Pandas vous retourne peuvent ne pas être cohérents ! Vérifiez avec la description des données fournie pour détecter si des colonnes devraient plutôt avoir un autre type.

Q2. L'encodage ordinal est un moyen rapide de tester un algorithme (prototypage), mais qui montre vite ses limites, surtout pour les données sans relation d'ordre. Quels autres encodages pouvez-vous utiliser ?

Astuce : il vaut peut-être le coup d'essayer plusieurs formes d'encodages pour voir l'impact sur les performances...

On rappelle quelques méthodes d'encodage :

- Encodage ordinal
- Encodage one-hot
- Encodage binaire

Q3. Quelles sont les échelles de chaque colonne numérique (valeurs min vs max) ? Quelles sont les distributions de ces colonnes ? Quelles normalisations pouvez-vous utiliser ?

Astuce : il vaut peut-être le coup d'essayer plusieurs formes de normalisations pour voir l'impact sur les performances...

On rappelle quelques méthodes de normalisation :
- Linéaire (min-max)
- Z-score
- Log

Vous pouvez utiliser du *clipping* en plus de chacune des méthodes de normalisation.

Q4. Existe-t-il des données manquantes ? Quelles colonnes ont beaucoup de données manquantes ? Combien de lignes possèdent au moins une valeur manquante ?

**Attention** : si votre fonction de préparation de données supprime les lignes contenant 1 valeur manquante (NA), vous allez supprimer des lignes du jeu de données "de déploiement" (ou d'utilisation). Ce qui veut dire que vous ne ferez pas de prédictions sur ces lignes, ce qui baissera forcément votre score ! Durant la phase d'entraînement, on peut se permettre de supprimer les valeurs manquantes si on pense qu'elles vont empêcher l'apprentissage ; mais, en déploiement, quand votre modèle sera utilisé par des personnes extérieures, vous pouvez difficilement vous permettre de rejeter des entrées... (Ou alors, vous risquez que vos utilisateurs finissent par ne plus utiliser votre modèle)

Vous pouvez essayer de supprimer les données manquantes, ou de les remplacer...

On rappelle quelques méthodes de remplacement :
- Par un valeur "sentinelle" indiquant une absence (par exemple, `-1`)
- Par moyenne, médiane, ou mode (valeur fixe)
- Par *forward-fill* ou *backward-fill*
- Par apprentissage non-supervisé

Les méthodes par valeur fixe (moyenne, médiane, mode) ou par données existantes (forward-fill, backward-fill) peuvent être utilisées en corrélation avec d'autres colonnes, par exemple en faisant un `groupby`.

Q5. On a essayé une Régression Logistique, mais il existe d'autres algorithmes de classification ! Lesquels paraissent intéressants pour cette tâche ?
Comparez ces algorithmes, sur vos différentes façons de préparer vos données.

Q6. Chaque algorithme possède un certain nombre d'hyperparamètres, impactant les performances. Comparez les performances de différents hyperparamètres, pour chaque modèle.

La documentation de Scikit-learn liste les hyperparamètres dans le constructeur de chaque algorithme. Attention : cette liste est souvent plus étoffée que celle vue en cours, car Scikit propose des optimisations, mais laisse la configuration de ces optimisations au choix des utilisateurs...

Q7. Quelle est la fréquence de chaque classe ? Vous pouvez essayer de ré-équilibrer les données pour améliorer les performances.

On rappelle quelques méthodes de ré-équilibrage :
- *Downsampling* : diminution de la classe majoritaire jusqu'à atteindre la même proportion que la classe minoritaire.
- *Upsampling* : répétition des individus de la classe minoritaire jusqu'à atteindre la même proportion que la classe majoritaire.
- *Loss weighting* : on peut pondérer la fonction de *loss* pour rendre les erreurs sur la classe minoritaire plus coûteuses. Attention : cela suppose que la distribution des classes dans les données d'utilisation seront les mêmes ! Dans la pratique, vous ne pouvez pas en être certains...
- Génération de données : on peut générer des données similaires aux données existantes, en espérent qu'elles soient suffisamment proches pour qu'elles représentent bien le phénomène que l'on cherche à modéliser. Attention : dans la pratique, c'est compliqué de générer des données pour modéliser un phénomène, sans avoir un modèle du phénomène ...

Q7. Dans la fonction `evaluate_model()`, on vous fait initialement mesurer les performances de votre modèle sur les données de test. Or, en comparant les modèles sur ces données, on risque de choisir un modèle qui est trop spécialisé sur ces données, mais qui ne saura pas généraliser aux données d'utilisation ! On a vu en cours une technique pour éviter ce problème : la *cross-validation*. Vous pouvez implémenter une autre fonction `evaluate_model_cv()` qui suit le même principe mais qui utilise la *cross-validation* pour comparer les modèles.

Astuce : votre fonction peut évaluer plusieurs modèles et retourner un *DataFrame* contenant plusieurs résultats, avec `pd.DataFrame([ {...}, {...}, {...} ])`, où chaque `{...}` est un dictionnaire comme celui de la fonction `evalute_model()`. Cela permettrait d'évaluer les modèles sur les mêmes *folds* de la *cross-validation*.

Q8. Vous pouvez essayer des méthodes ensemblistes.

On rappelle les 3 principales catégories de méthodes ensemblistes vues en cours :
- *Bagging*
- *Boosting*
- *Stacking*

In [333]:
# Charger les données de prototypage
df = pd.read_csv('data_prototyping.csv', na_values='?')

# Vérifier les types de données
print(df.dtypes)

age                                           int64
sex                                          object
chest pain                                   object
resting blood pressure (mm Hg)              float64
cholestoral (mg/dl)                         float64
fasting blood sugar > 120 mg/dl              object
resting electrocardiograph                   object
max heart rate                              float64
exercise induced angina                      object
oldpeak                                     float64
slope                                        object
number of colored vessels by fluoroscopy    float64
thalassemia                                 float64
class                                        object
dtype: object


In [334]:
# Identifier les colonnes numériques et catégorielles
categorical_columns = []
numerical_columns = []

for col in df.columns:
    if df[col].dtype == 'object':
        categorical_columns.append(col)
    else:
        numerical_columns.append(col)

print("Colonnes catégorielles :", categorical_columns)
print("Colonnes numériques :", numerical_columns)


Colonnes catégorielles : ['sex', 'chest pain', 'fasting blood sugar > 120 mg/dl', 'resting electrocardiograph', 'exercise induced angina', 'slope', 'class']
Colonnes numériques : ['age', 'resting blood pressure (mm Hg)', 'cholestoral (mg/dl)', 'max heart rate', 'oldpeak', 'number of colored vessels by fluoroscopy', 'thalassemia']


In [335]:
for col in categorical_columns:
    print(f"\nValeurs uniques pour la colonne '{col}':")
    print(df[col].unique())



Valeurs uniques pour la colonne 'sex':
['male' 'female']

Valeurs uniques pour la colonne 'chest pain':
['asymptomatic' 'non-anginal pain' 'atypical angina' 'typical angina']

Valeurs uniques pour la colonne 'fasting blood sugar > 120 mg/dl':
['no' 'yes' nan]

Valeurs uniques pour la colonne 'resting electrocardiograph':
['normal' 'abnormal' 'hypertrophy']

Valeurs uniques pour la colonne 'exercise induced angina':
['yes' 'no' nan]

Valeurs uniques pour la colonne 'slope':
['downsloping' nan 'flat' 'upsloping']

Valeurs uniques pour la colonne 'class':
['disease' 'no disease']


In [336]:
missing_values = df.isnull().sum()
print("\nValeurs manquantes par colonne :")
print(missing_values)



Valeurs manquantes par colonne :
age                                           0
sex                                           0
chest pain                                    0
resting blood pressure (mm Hg)               25
cholestoral (mg/dl)                          22
fasting blood sugar > 120 mg/dl              23
resting electrocardiograph                    0
max heart rate                               25
exercise induced angina                      25
oldpeak                                      27
slope                                       198
number of colored vessels by fluoroscopy    284
thalassemia                                 253
class                                         0
dtype: int64


In [337]:
def prepare_data_v2(df):
    # Importer les bibliothèques nécessaires
    import pandas as pd
    import numpy as np

    # Copier le DataFrame pour éviter de le modifier directement
    df = df.copy()

    # Remplacer les '?' par NaN
    df.replace('?', np.nan, inplace=True)

    # Gestion des valeurs manquantes

    ## Supprimer les colonnes avec trop de valeurs manquantes
    # D'après l'EDA, nous pouvons supprimer 'number of colored vessels by fluoroscopy' et 'thalassemia'
    df = df.drop(columns=['number of colored vessels by fluoroscopy', 'thalassemia'])

    ## Imputer les valeurs manquantes pour les colonnes numériques
    numerical_columns = [
        'age', 'resting blood pressure (mm Hg)', 'cholestoral (mg/dl)',
        'max heart rate', 'oldpeak'
    ]
    for col in numerical_columns:
        df[col] = df[col].astype(float)
        df[col].fillna(df[col].median(), inplace=True)

    ## Imputer les valeurs manquantes pour les colonnes catégorielles
    categorical_columns = [
        'sex', 'chest pain', 'fasting blood sugar > 120 mg/dl',
        'resting electrocardiograph', 'exercise induced angina', 'slope'
    ]
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Encodage des variables catégorielles

    ## Encodage binaire
    df['sex'] = df['sex'].map({'male': 1, 'female': 0})
    df['fasting blood sugar > 120 mg/dl'] = df['fasting blood sugar > 120 mg/dl'].map({'yes': 1, 'no': 0})
    df['exercise induced angina'] = df['exercise induced angina'].map({'yes': 1, 'no': 0})

    ## Encodage one-hot pour les colonnes avec plus de deux catégories
    df = pd.get_dummies(df, columns=['chest pain', 'resting electrocardiograph', 'slope'], drop_first=True)

    # S'assurer que la colonne cible est présente et non modifiée
    if 'class' not in df.columns:
        raise KeyError("'class' column not found in DataFrame")

    return df



In [338]:
# Séparer les données en df_train et df_test avec stratification
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)

# Préparer les données avec prepare_data_v2
df_train_v2 = prepare_data_v2(df_train)
df_test_v2 = prepare_data_v2(df_test)

# Afficher un aperçu des données préparées
print(df_train_v2.head())
print(df_test_v2.head())



      age  sex  resting blood pressure (mm Hg)  cholestoral (mg/dl)  \
109  51.0    0                           150.0                200.0   
270  55.0    1                           110.0                277.0   
194  49.0    1                           140.0                228.0   
357  32.0    1                           125.0                254.0   
164  58.0    1                           105.0                240.0   

     fasting blood sugar > 120 mg/dl  max heart rate  exercise induced angina  \
109                                0           120.0                        0   
270                                0           160.0                        0   
194                                0           130.0                        0   
357                                0           155.0                        0   
164                                0           154.0                        1   

     oldpeak       class  chest pain_atypical angina  \
109      0.5  no disease      

/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3911969470.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3911969470.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [339]:
# Importer le modèle de régression logistique
from sklearn.linear_model import LogisticRegression

# Entraîner et évaluer le modèle sur les données préparées avec v2
results_v2 = evaluate_model(
    df_train_v2,
    df_test_v2,
    'v2',
    'LogisticRegression',
    LogisticRegression,
    {'penalty': None, 'max_iter': 1000}
)

# Afficher les résultats
print(results_v2)

# Enregistrer les résultats
append_results(results_v2)


            ModelName Data                      Hyperparameters  RandomSeed  \
0  LogisticRegression   v2  {'penalty': None, 'max_iter': 1000}  3362672484   

   Score_f2  Score_precision  Score_recall  Score_accuracy  \
0  0.365854              0.6      0.333333        0.826087   

                                  TrainedEstimator  
0  LogisticRegression(max_iter=1000, penalty=None)  


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [340]:
from sklearn.tree import DecisionTreeClassifier

# Entraîner et évaluer le modèle
results_dt = evaluate_model(
    df_train_v2,
    df_test_v2,
    'v2',
    'DecisionTreeClassifier',
    DecisionTreeClassifier,
    {'max_depth': None, 'random_state': 42}
)

append_results(results_dt)


In [341]:
from sklearn.ensemble import RandomForestClassifier

results_rf = evaluate_model(
    df_train_v2,
    df_test_v2,
    'v2',
    'RandomForestClassifier',
    RandomForestClassifier,
    {'n_estimators': 100, 'random_state': 42}
)

append_results(results_rf)

In [342]:
results_lr_l2 = evaluate_model(
    df_train_v2,
    df_test_v2,
    'v2',
    'LogisticRegression_L2',
    LogisticRegression,
    {'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear', 'max_iter': 1000}
)

append_results(results_lr_l2)


In [343]:
# Essayer différentes valeurs pour max_depth
for depth in [3, 5, 7, None]:
    results_dt = evaluate_model(
        df_train_v2,
        df_test_v2,
        'v2',
        f'DecisionTreeClassifier_max_depth_{depth}',
        DecisionTreeClassifier,
        {'max_depth': depth, 'random_state': 42}
    )
    append_results(results_dt)


In [344]:
def prepare_data_v3(df):
    # Importer les bibliothèques nécessaires
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler

    # Copier le DataFrame pour éviter de le modifier directement
    df = df.copy()

    # Remplacer les '?' par NaN
    df.replace('?', np.nan, inplace=True)

    # Gestion des valeurs manquantes

    # Supprimer les colonnes avec trop de valeurs manquantes
    df = df.drop(columns=['number of colored vessels by fluoroscopy', 'thalassemia'])

    # Imputer les valeurs manquantes pour les colonnes numériques
    numerical_columns = [
        'age', 'resting blood pressure (mm Hg)', 'cholestoral (mg/dl)',
        'max heart rate', 'oldpeak'
    ]
    for col in numerical_columns:
        df[col] = df[col].astype(float)
        df[col].fillna(df[col].median(), inplace=True)

    # Imputer les valeurs manquantes pour les colonnes catégorielles
    categorical_columns = [
        'sex', 'chest pain', 'fasting blood sugar > 120 mg/dl',
        'resting electrocardiograph', 'exercise induced angina', 'slope'
    ]
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Encodage des variables catégorielles

    # Encodage binaire
    df['sex'] = df['sex'].map({'male': 1, 'female': 0})
    df['fasting blood sugar > 120 mg/dl'] = df['fasting blood sugar > 120 mg/dl'].map({'yes': 1, 'no': 0})
    df['exercise induced angina'] = df['exercise induced angina'].map({'yes': 1, 'no': 0})

    # Encodage one-hot pour les colonnes avec plus de deux catégories
    df = pd.get_dummies(df, columns=['chest pain', 'resting electrocardiograph', 'slope'], drop_first=True)

    # Normalisation des variables numériques
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    # S'assurer que la colonne cible est présente et non modifiée
    if 'class' not in df.columns:
        raise KeyError("'class' column not found in DataFrame")

    return df


In [345]:
# Préparer les données avec prepare_data_v3
df_train_v3 = prepare_data_v3(df_train)
df_test_v3 = prepare_data_v3(df_test)

# Afficher un aperçu des données préparées
print(df_train_v3.head())
print(df_test_v3.head())


/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3968901949.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/g9/jzqz957j1slb_dkhyp9fl5_w0000gn/T/ipykernel_66927/3968901949.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

          age  sex  resting blood pressure (mm Hg)  cholestoral (mg/dl)  \
109 -0.097300    0                        1.030776            -0.270998   
270  0.322807    1                       -1.209643             0.652558   
194 -0.307354    1                        0.470671             0.064841   
357 -2.092809    1                       -0.369486             0.376691   
164  0.637887    1                       -1.489695             0.208772   

     fasting blood sugar > 120 mg/dl  max heart rate  exercise induced angina  \
109                                0       -0.972118                        0   
270                                0        0.663398                        0   
194                                0       -0.563239                        0   
357                                0        0.458958                        0   
164                                0        0.418070                        1   

      oldpeak       class  chest pain_atypical angina  \
109 -

In [346]:
from sklearn.linear_model import LogisticRegression

# Entraîner et évaluer le modèle
results_lr_v3 = evaluate_model(
    df_train_v3,
    df_test_v3,
    'v3',
    'LogisticRegression_L2_Normalized',
    LogisticRegression,
    {'penalty': 'l2', 'C': 1.0, 'solver': 'liblinear', 'max_iter': 1000}
)

# Enregistrer les résultats
append_results(results_lr_v3)

# Afficher les résultats
print(results_lr_v3)


                          ModelName Data  \
0  LogisticRegression_L2_Normalized   v3   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'penalty': 'l2', 'C': 1.0, 'solver': 'libline...   190479754  0.301205   

   Score_precision  Score_recall  Score_accuracy  \
0         0.454545      0.277778        0.793478   

                                    TrainedEstimator  
0  LogisticRegression(max_iter=1000, solver='libl...  


In [347]:
# Entraîner et évaluer le modèle avec max_depth=5 (meilleur jusqu'à présent)
results_dt_v3 = evaluate_model(
    df_train_v3,
    df_test_v3,
    'v3',
    'DecisionTreeClassifier_max_depth_5_Normalized',
    DecisionTreeClassifier,
    {'max_depth': 5, 'random_state': 42}
)

# Enregistrer les résultats
append_results(results_dt_v3)

# Afficher les résultats
print(results_dt_v3)

                                       ModelName Data  \
0  DecisionTreeClassifier_max_depth_5_Normalized   v3   

                        Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 5, 'random_state': 42}  3786572474  0.483871   

   Score_precision  Score_recall  Score_accuracy  \
0         0.428571           0.5        0.771739   

                                    TrainedEstimator  
0  DecisionTreeClassifier(max_depth=5, random_sta...  


In [348]:
results_dt_v3_optimized = evaluate_model(
    df_train_v3,
    df_test_v3,
    'v3',
    'DecisionTreeClassifier_Optimized',
    DecisionTreeClassifier,
    {
        'max_depth': 5,
        'min_samples_split': 5,
        'min_samples_leaf': 2,
        'criterion': 'entropy',
        'random_state': 42
    }
)

append_results(results_dt_v3_optimized)

In [349]:
def prepare_data_v4(df):
    import pandas as pd
    import numpy as np
    from sklearn.experimental import enable_iterative_imputer  # Nécessaire pour utiliser IterativeImputer
    from sklearn.impute import IterativeImputer
    from sklearn.preprocessing import StandardScaler

    # Copier le DataFrame pour éviter de le modifier directement
    df = df.copy()

    # Remplacer les '?' par NaN
    df.replace('?', np.nan, inplace=True)

    # Supprimer les colonnes avec trop de valeurs manquantes
    # On peut tenter d'imputer 'thalassemia' si on pense que c'est une variable importante
    df = df.drop(columns=['number of colored vessels by fluoroscopy'])

    # Encodage des variables catégorielles

    # Encodage binaire
    df['sex'] = df['sex'].map({'male': 1, 'female': 0})
    df['fasting blood sugar > 120 mg/dl'] = df['fasting blood sugar > 120 mg/dl'].map({'yes': 1, 'no': 0})
    df['exercise induced angina'] = df['exercise induced angina'].map({'yes': 1, 'no': 0})

    # Gérer les valeurs manquantes dans les colonnes catégorielles
    categorical_columns = ['sex', 'fasting blood sugar > 120 mg/dl', 'exercise induced angina',
                           'chest pain', 'resting electrocardiograph', 'slope', 'thalassemia']
    for col in categorical_columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna(df[col].mode()[0])

    # Encodage one-hot pour les variables catégorielles restantes
    df = pd.get_dummies(df, columns=['chest pain', 'resting electrocardiograph', 'slope', 'thalassemia'], drop_first=True)

    # Séparer la cible
    y = df['class']
    X = df.drop(columns=['class'])

    # Imputation multiple avec IterativeImputer
    imputer = IterativeImputer(random_state=42)
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    # Normalisation des variables numériques
    numerical_columns = [
        'age', 'resting blood pressure (mm Hg)', 'cholestoral (mg/dl)',
        'max heart rate', 'oldpeak'
    ]
    scaler = StandardScaler()
    X_imputed[numerical_columns] = scaler.fit_transform(X_imputed[numerical_columns])

    # Reconstituer le DataFrame
    df_prepared = pd.concat([X_imputed, y.reset_index(drop=True)], axis=1)

    return df_prepared


In [350]:
def downsample_data(X, y):
    from sklearn.utils import resample
    
    # Combiner X et y
    df = pd.concat([X, y], axis=1)
    
    # Séparer les classes majoritaire et minoritaire
    df_majority = df[df['class'] == 'no disease']
    df_minority = df[df['class'] == 'disease']
    
    # Downsample de la classe majoritaire
    df_majority_downsampled = resample(
        df_majority,
        replace=False,
        n_samples=len(df_minority),
        random_state=42
    )
    
    # Combiner les deux classes
    df_downsampled = pd.concat([df_majority_downsampled, df_minority])
    
    # Séparer X et y
    X_downsampled = df_downsampled.drop(columns=['class'])
    y_downsampled = df_downsampled['class']
    
    return X_downsampled, y_downsampled


In [351]:
# Séparer les données en df_train et df_test avec stratification
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)

# Préparer les données avec prepare_data_v4
df_train_v4 = prepare_data_v4(df_train)
df_test_v4 = prepare_data_v4(df_test)


In [352]:
# Séparer X et y
X_train_v4 = df_train_v4.drop(columns=['class'])
y_train_v4 = df_train_v4['class']
X_test_v4 = df_test_v4.drop(columns=['class'])
y_test_v4 = df_test_v4['class']

# Appliquer le downsampling
X_train_downsampled, y_train_downsampled = downsample_data(X_train_v4, y_train_v4)

In [353]:
# Recréer df_train_downsampled
df_train_downsampled = pd.concat([X_train_downsampled, y_train_downsampled], axis=1)

# Entraîner et évaluer le modèle
from sklearn.tree import DecisionTreeClassifier

results_downsampled = evaluate_model(
    df_train_downsampled,
    df_test_v4,
    'v4_downsampled',
    'DecisionTreeClassifier_Downsampled',
    DecisionTreeClassifier,
    {'max_depth': 5, 'random_state': 42}
)

# Enregistrer les résultats
append_results(results_downsampled)

# Afficher les résultats
print(results_downsampled)

                            ModelName            Data  \
0  DecisionTreeClassifier_Downsampled  v4_downsampled   

                        Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 5, 'random_state': 42}  3177593907  0.490196   

   Score_precision  Score_recall  Score_accuracy  \
0         0.333333      0.555556        0.695652   

                                    TrainedEstimator  
0  DecisionTreeClassifier(max_depth=5, random_sta...  


In [354]:
def upsample_data(X, y):
    from sklearn.utils import resample
    
    # Combiner X et y
    df = pd.concat([X, y], axis=1)
    
    # Séparer les classes majoritaire et minoritaire
    df_majority = df[df['class'] == 'no disease']
    df_minority = df[df['class'] == 'disease']
    
    # Upsample de la classe minoritaire
    df_minority_upsampled = resample(
        df_minority,
        replace=True,
        n_samples=len(df_majority),
        random_state=42
    )
    
    # Combiner les deux classes
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])
    
    # Séparer X et y
    X_upsampled = df_upsampled.drop(columns=['class'])
    y_upsampled = df_upsampled['class']
    
    return X_upsampled, y_upsampled

In [355]:
# Appliquer l'upsampling
X_train_upsampled, y_train_upsampled = upsample_data(X_train_v4, y_train_v4)

In [356]:
# Recréer df_train_upsampled
df_train_upsampled = pd.concat([X_train_upsampled, y_train_upsampled], axis=1)

# Entraîner et évaluer le modèle
results_upsampled = evaluate_model(
    df_train_upsampled,
    df_test_v4,
    'v4_upsampled',
    'DecisionTreeClassifier_Upsampled',
    DecisionTreeClassifier,
    {'max_depth': 5, 'random_state': 42}
)

# Enregistrer les résultats
append_results(results_upsampled)

# Afficher les résultats
print(results_upsampled)

                          ModelName          Data  \
0  DecisionTreeClassifier_Upsampled  v4_upsampled   

                        Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 5, 'random_state': 42}  2583452187  0.436893   

   Score_precision  Score_recall  Score_accuracy  \
0         0.290323           0.5        0.663043   

                                    TrainedEstimator  
0  DecisionTreeClassifier(max_depth=5, random_sta...  


In [357]:
# Définir les poids de classe
class_weights = {'no disease': 1, 'disease': 3}  # Vous pouvez ajuster les poids selon les besoins

# Entraîner et évaluer le modèle
results_weighted = evaluate_model(
    df_train_v4,
    df_test_v4,
    'v4_weighted',
    'DecisionTreeClassifier_Weighted',
    DecisionTreeClassifier,
    {'max_depth': 5, 'random_state': 42, 'class_weight': class_weights}
)

# Enregistrer les résultats
append_results(results_weighted)

# Afficher les résultats
print(results_weighted)

                         ModelName         Data  \
0  DecisionTreeClassifier_Weighted  v4_weighted   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 5, 'random_state': 42, 'class_we...  2093047478  0.360825   

   Score_precision  Score_recall  Score_accuracy  \
0             0.28      0.388889        0.684783   

                                    TrainedEstimator  
0  DecisionTreeClassifier(class_weight={'disease'...  


In [358]:
from imblearn.over_sampling import SMOTE

def smote_data(X, y):
    smote = SMOTE(random_state=42)
    X_smote, y_smote = smote.fit_resample(X, y)
    return X_smote, y_smote

In [359]:
# Appliquer SMOTE
X_train_smote, y_train_smote = smote_data(X_train_v4, y_train_v4)

In [360]:
# Recréer df_train_smote
df_train_smote = pd.concat([X_train_smote, y_train_smote], axis=1)

# Entraîner et évaluer le modèle
results_smote = evaluate_model(
    df_train_smote,
    df_test_v4,
    'v4_smote',
    'DecisionTreeClassifier_SMOTE',
    DecisionTreeClassifier,
    {'max_depth': 5, 'random_state': 42}
)

# Enregistrer les résultats
append_results(results_smote)

# Afficher les résultats
print(results_smote)

                      ModelName      Data  \
0  DecisionTreeClassifier_SMOTE  v4_smote   

                        Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 5, 'random_state': 42}  3058381753  0.376344   

   Score_precision  Score_recall  Score_accuracy  \
0         0.333333      0.388889        0.728261   

                                    TrainedEstimator  
0  DecisionTreeClassifier(max_depth=5, random_sta...  


In [361]:
def optimize_hyperparameters(model_class, param_grid, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import make_scorer, fbeta_score

    # Créer un scorer personnalisé pour le F-beta score avec beta=2
    f2_scorer = make_scorer(fbeta_score, beta=2, pos_label='disease')

    grid_search = GridSearchCV(
        estimator=model_class(),
        param_grid=param_grid,
        scoring=f2_scorer,
        refit=True,
        cv=5,
        n_jobs=-1
    )

    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_

In [362]:
# Définir la grille de paramètres pour LogisticRegression
param_grid_lr = {
    'penalty': ['l2'],  # 'liblinear' supporte 'l1' et 'l2'
    'C': [0.1, 1, 10],
    'solver': ['liblinear'],
    'max_iter': [1000]
}

# Optimiser les hyperparamètres
best_estimator_lr_down, best_params_lr_down = optimize_hyperparameters(
    LogisticRegression,
    param_grid_lr,
    X_train_downsampled,
    y_train_downsampled
)

print("Meilleurs paramètres pour LogisticRegression avec Downsampling :", best_params_lr_down)

# Évaluer le modèle optimisé
results_lr_down = evaluate_model(
    df_train_downsampled,
    df_test_v4,
    'v4_downsampled',
    'LogisticRegression_Downsampled',
    LogisticRegression,
    best_params_lr_down
)

# Enregistrer les résultats
append_results(results_lr_down)

# Afficher les résultats
print(results_lr_down)


Meilleurs paramètres pour LogisticRegression avec Downsampling : {'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
                        ModelName            Data  \
0  LogisticRegression_Downsampled  v4_downsampled   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', ...   381573423       0.6   

   Score_precision  Score_recall  Score_accuracy  \
0         0.428571      0.666667         0.76087   

                                    TrainedEstimator  
0  LogisticRegression(C=0.1, max_iter=1000, solve...  


In [363]:
# Optimiser les hyperparamètres
best_estimator_lr_up, best_params_lr_up = optimize_hyperparameters(
    LogisticRegression,
    param_grid_lr,
    X_train_upsampled,
    y_train_upsampled
)

print("Meilleurs paramètres pour LogisticRegression avec Upsampling :", best_params_lr_up)

# Évaluer le modèle optimisé
results_lr_up = evaluate_model(
    df_train_upsampled,
    df_test_v4,
    'v4_upsampled',
    'LogisticRegression_Upsampled',
    LogisticRegression,
    best_params_lr_up
)

# Enregistrer les résultats
append_results(results_lr_up)

# Afficher les résultats
print(results_lr_up)


Meilleurs paramètres pour LogisticRegression avec Upsampling : {'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
                      ModelName          Data  \
0  LogisticRegression_Upsampled  v4_upsampled   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'C': 10, 'max_iter': 1000, 'penalty': 'l2', '...  3412526607  0.549451   

   Score_precision  Score_recall  Score_accuracy  \
0         0.526316      0.555556        0.815217   

                                    TrainedEstimator  
0  LogisticRegression(C=10, max_iter=1000, solver...  


In [364]:
# Définir les poids de classe
class_weights_lr = {'no disease': 1, 'disease': 3}

# Définir la grille de paramètres avec class_weight
param_grid_lr_weighted = {
    'penalty': ['l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear'],
    'max_iter': [1000],
    'class_weight': [class_weights_lr]
}

# Optimiser les hyperparamètres
best_estimator_lr_weighted, best_params_lr_weighted = optimize_hyperparameters(
    LogisticRegression,
    param_grid_lr_weighted,
    X_train_v4,
    y_train_v4
)

print("Meilleurs paramètres pour LogisticRegression avec Loss Weighting :", best_params_lr_weighted)

# Évaluer le modèle optimisé
df_train_weighted = pd.concat([X_train_v4, y_train_v4], axis=1)
results_lr_weighted = evaluate_model(
    df_train_weighted,
    df_test_v4,
    'v4_weighted',
    'LogisticRegression_Weighted',
    LogisticRegression,
    best_params_lr_weighted
)

# Enregistrer les résultats
append_results(results_lr_weighted)

# Afficher les résultats
print(results_lr_weighted)


Meilleurs paramètres pour LogisticRegression avec Loss Weighting : {'C': 0.1, 'class_weight': {'no disease': 1, 'disease': 3}, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
                     ModelName         Data  \
0  LogisticRegression_Weighted  v4_weighted   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'C': 0.1, 'class_weight': {'no disease': 1, '...   590938642  0.531915   

   Score_precision  Score_recall  Score_accuracy  \
0         0.454545      0.555556        0.782609   

                                    TrainedEstimator  
0  LogisticRegression(C=0.1, class_weight={'disea...  


In [365]:
# Optimiser les hyperparamètres
best_estimator_lr_smote, best_params_lr_smote = optimize_hyperparameters(
    LogisticRegression,
    param_grid_lr,
    X_train_smote,
    y_train_smote
)

print("Meilleurs paramètres pour LogisticRegression avec SMOTE :", best_params_lr_smote)

# Évaluer le modèle optimisé
results_lr_smote = evaluate_model(
    df_train_smote,
    df_test_v4,
    'v4_smote',
    'LogisticRegression_SMOTE',
    LogisticRegression,
    best_params_lr_smote
)

# Enregistrer les résultats
append_results(results_lr_smote)

# Afficher les résultats
print(results_lr_smote)


Meilleurs paramètres pour LogisticRegression avec SMOTE : {'C': 1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
                  ModelName      Data  \
0  LogisticRegression_SMOTE  v4_smote   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'C': 1, 'max_iter': 1000, 'penalty': 'l2', 's...  3337360932   0.48913   

   Score_precision  Score_recall  Score_accuracy  \
0             0.45           0.5        0.782609   

                                    TrainedEstimator  
0  LogisticRegression(C=1, max_iter=1000, solver=...  


In [366]:
# Définir la grille de paramètres pour RandomForestClassifier
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'random_state': [42]
}

# Optimiser les hyperparamètres
best_estimator_rf_down, best_params_rf_down = optimize_hyperparameters(
    RandomForestClassifier,
    param_grid_rf,
    X_train_downsampled,
    y_train_downsampled
)

print("Meilleurs paramètres pour RandomForestClassifier avec Downsampling :", best_params_rf_down)

# Évaluer le modèle optimisé
results_rf_down = evaluate_model(
    df_train_downsampled,
    df_test_v4,
    'v4_downsampled',
    'RandomForestClassifier_Downsampled',
    RandomForestClassifier,
    best_params_rf_down
)

# Enregistrer les résultats
append_results(results_rf_down)

# Afficher les résultats
print(results_rf_down)


Meilleurs paramètres pour RandomForestClassifier avec Downsampling : {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
                            ModelName            Data  \
0  RandomForestClassifier_Downsampled  v4_downsampled   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': None, 'max_features': 'sqrt', 'm...  1603038928  0.576923   

   Score_precision  Score_recall  Score_accuracy  \
0            0.375      0.666667        0.717391   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(max_features='sqrt', r...  


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
92 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/homebrew/lib/python3.11/site-package

In [367]:
from sklearn.ensemble import RandomForestClassifier

# Définir la grille de paramètres pour RandomForestClassifier
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'random_state': [42]
}

# Optimiser les hyperparamètres
best_estimator_rf_down, best_params_rf_down = optimize_hyperparameters(
    RandomForestClassifier,
    param_grid_rf,
    X_train_downsampled,
    y_train_downsampled
)

print("Meilleurs paramètres pour RandomForestClassifier avec Downsampling :", best_params_rf_down)

# Évaluer le modèle optimisé
results_rf_down = evaluate_model(
    df_train_downsampled,
    df_test_v4,
    'v4_downsampled',
    'RandomForestClassifier_Downsampled',
    RandomForestClassifier,
    best_params_rf_down
)

# Enregistrer les résultats
append_results(results_rf_down)

# Afficher les résultats
print(results_rf_down)


Meilleurs paramètres pour RandomForestClassifier avec Downsampling : {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
                            ModelName            Data  \
0  RandomForestClassifier_Downsampled  v4_downsampled   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': None, 'max_features': 'sqrt', 'm...  1102679409  0.576923   

   Score_precision  Score_recall  Score_accuracy  \
0            0.375      0.666667        0.717391   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(max_features='sqrt', r...  


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/homebrew/lib/python3.11/site-package

In [368]:
# Appliquer Upsampling
X_train_upsampled, y_train_upsampled = upsample_data(X_train_v4, y_train_v4)
df_train_upsampled = pd.concat([X_train_upsampled, y_train_upsampled], axis=1)

# Optimiser les hyperparamètres
best_estimator_rf_up, best_params_rf_up = optimize_hyperparameters(
    RandomForestClassifier,
    param_grid_rf,
    X_train_upsampled,
    y_train_upsampled
)

print("Meilleurs paramètres pour RandomForestClassifier avec Upsampling :", best_params_rf_up)

# Évaluer le modèle optimisé
results_rf_up = evaluate_model(
    df_train_upsampled,
    df_test_v4,
    'v4_upsampled',
    'RandomForestClassifier_Upsampled',
    RandomForestClassifier,
    best_params_rf_up
)

# Enregistrer les résultats
append_results(results_rf_up)

# Afficher les résultats
print(results_rf_up)


Meilleurs paramètres pour RandomForestClassifier avec Upsampling : {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
                          ModelName          Data  \
0  RandomForestClassifier_Upsampled  v4_upsampled   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': 10, 'max_features': 'sqrt', 'min...  3830736073  0.666667   

   Score_precision  Score_recall  Score_accuracy  \
0         0.666667      0.666667        0.869565   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(max_depth=10, max_feat...  


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/homebrew/lib/python3.11/site-package

In [369]:
# Définir les poids de classe pour RandomForestClassifier
class_weights_rf = {'no disease': 1, 'disease': 3}

# Définir la grille de paramètres avec class_weight
param_grid_rf_weighted = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'random_state': [42],
    'class_weight': [class_weights_rf]
}

# Optimiser les hyperparamètres
best_estimator_rf_weighted, best_params_rf_weighted = optimize_hyperparameters(
    RandomForestClassifier,
    param_grid_rf_weighted,
    X_train_v4,
    y_train_v4
)

print("Meilleurs paramètres pour RandomForestClassifier avec Loss Weighting :", best_params_rf_weighted)

# Évaluer le modèle optimisé
df_train_weighted_rf = pd.concat([X_train_v4, y_train_v4], axis=1)
results_rf_weighted = evaluate_model(
    df_train_weighted_rf,
    df_test_v4,
    'v4_weighted',
    'RandomForestClassifier_Weighted',
    RandomForestClassifier,
    best_params_rf_weighted
)

# Enregistrer les résultats
append_results(results_rf_weighted)

# Afficher les résultats
print(results_rf_weighted)


Meilleurs paramètres pour RandomForestClassifier avec Loss Weighting : {'class_weight': {'no disease': 1, 'disease': 3}, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100, 'random_state': 42}
                         ModelName         Data  \
0  RandomForestClassifier_Weighted  v4_weighted   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'class_weight': {'no disease': 1, 'disease': ...   803945210  0.465116   

   Score_precision  Score_recall  Score_accuracy  \
0         0.571429      0.444444        0.826087   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(max_features='sqrt', m...  


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
68 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/homebrew/lib/python3.11/site-package

In [370]:
# Optimiser les hyperparamètres
best_estimator_rf_smote, best_params_rf_smote = optimize_hyperparameters(
    RandomForestClassifier,
    param_grid_rf,
    X_train_smote,
    y_train_smote
)

print("Meilleurs paramètres pour RandomForestClassifier avec SMOTE :", best_params_rf_smote)

# Évaluer le modèle optimisé
results_rf_smote = evaluate_model(
    df_train_smote,
    df_test_v4,
    'v4_smote',
    'RandomForestClassifier_SMOTE',
    RandomForestClassifier,
    best_params_rf_smote
)

# Enregistrer les résultats
append_results(results_rf_smote)

# Afficher les résultats
print(results_rf_smote)


Meilleurs paramètres pour RandomForestClassifier avec SMOTE : {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
                      ModelName      Data  \
0  RandomForestClassifier_SMOTE  v4_smote   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'max_depth': None, 'max_features': 'sqrt', 'm...  1149985292       0.5   

   Score_precision  Score_recall  Score_accuracy  \
0              0.5           0.5        0.804348   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(max_features='sqrt', r...  


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
87 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/homebrew/lib/python3.11/site-package

In [371]:
from sklearn.ensemble import BaggingClassifier

param_grid_bagging = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0],
    'bootstrap': [True, False],
    'random_state': [42]
}


In [372]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score

def optimize_hyperparameters(model_class, param_grid, X_train, y_train):
    """
    Optimise les hyperparamètres d'un modèle en utilisant GridSearchCV avec un scorer personnalisé F2.
    
    Parameters:
        model_class: La classe du modèle (e.g., DecisionTreeClassifier).
        param_grid (dict): Grille des hyperparamètres à tester.
        X_train (DataFrame): Variables explicatives d'entraînement.
        y_train (Series): Variable cible d'entraînement.
    
    Returns:
        best_estimator_: Le meilleur modèle trouvé.
        best_params_: Les meilleurs hyperparamètres trouvés.
    """
    # Créer un scorer personnalisé pour le F2-score
    f2_scorer = make_scorer(fbeta_score, beta=2, pos_label='disease')
    
    grid_search = GridSearchCV(
        estimator=model_class(),
        param_grid=param_grid,
        scoring=f2_scorer,
        refit=True,
        cv=5,
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_


In [373]:
from imblearn.over_sampling import SMOTE

def smote_data(X, y):
    """
    Applique SMOTE pour générer des données synthétiques de la classe minoritaire.
    
    Parameters:
        X (DataFrame): Variables explicatives.
        y (Series): Variable cible.
    
    Returns:
        X_smote (DataFrame): Variables explicatives SMOTE.
        y_smote (Series): Variable cible SMOTE.
    """
    smote = SMOTE(random_state=42)
    X_smote, y_smote = smote.fit_resample(X, y)
    return X_smote, y_smote

# Appliquer SMOTE
X_train_smote_bagging, y_train_smote_bagging = smote_data(X_train_v4, y_train_v4)

In [374]:
# Optimiser les hyperparamètres pour BaggingClassifier
best_estimator_bagging, best_params_bagging = optimize_hyperparameters(
    BaggingClassifier,
    param_grid_bagging,
    X_train_smote_bagging,
    y_train_smote_bagging
)

print("Meilleurs paramètres pour BaggingClassifier avec SMOTE :", best_params_bagging)

# Recréer df_train_bagging_smote
df_train_bagging_smote = pd.concat([X_train_smote_bagging, y_train_smote_bagging], axis=1)

# Entraîner et évaluer le modèle
results_bagging = evaluate_model(
    df_train_bagging_smote,
    df_test_v4,
    'v4_bagging_smote',
    'BaggingClassifier_SMOTE',
    BaggingClassifier,
    best_params_bagging
)

# Enregistrer les résultats
append_results(results_bagging)

# Afficher les résultats
print(results_bagging)

Meilleurs paramètres pour BaggingClassifier avec SMOTE : {'bootstrap': True, 'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 50, 'random_state': 42}
                 ModelName              Data  \
0  BaggingClassifier_SMOTE  v4_bagging_smote   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'bootstrap': True, 'max_features': 1.0, 'max_...  1795035241  0.483871   

   Score_precision  Score_recall  Score_accuracy  \
0         0.428571           0.5        0.771739   

                                    TrainedEstimator  
0  (DecisionTreeClassifier(random_state=195292617...  


In [375]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid_boosting = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'random_state': [42]
}

In [376]:
# Optimiser les hyperparamètres pour GradientBoostingClassifier
best_estimator_boosting, best_params_boosting = optimize_hyperparameters(
    GradientBoostingClassifier,
    param_grid_boosting,
    X_train_smote,
    y_train_smote
)

print("Meilleurs paramètres pour GradientBoostingClassifier avec SMOTE :", best_params_boosting)

# Recréer df_train_boosting_smote
df_train_boosting_smote = pd.concat([X_train_smote, y_train_smote], axis=1)

# Entraîner et évaluer le modèle
results_boosting = evaluate_model(
    df_train_boosting_smote,
    df_test_v4,
    'v4_boosting_smote',
    'GradientBoostingClassifier_SMOTE',
    GradientBoostingClassifier,
    best_params_boosting
)

# Enregistrer les résultats
append_results(results_boosting)

# Afficher les résultats
print(results_boosting)

Meilleurs paramètres pour GradientBoostingClassifier avec SMOTE : {'learning_rate': 0.2, 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 42}
                          ModelName               Data  \
0  GradientBoostingClassifier_SMOTE  v4_boosting_smote   

                                     Hyperparameters  RandomSeed  Score_f2  \
0  {'learning_rate': 0.2, 'max_depth': 7, 'min_sa...  4176449941  0.494505   

   Score_precision  Score_recall  Score_accuracy  \
0         0.473684           0.5        0.793478   

                                    TrainedEstimator  
0  ([DecisionTreeRegressor(criterion='friedman_ms...  


## N'oubliez pas de mettre à jour vos fichiers `models_results.csv` et `predictions.csv` !!!

- `models_results.csv` à chaque expérimentation effectuée ;
- et `predictions.csv` une fois votre meilleur modèle identifié.

Ces fichiers sont à rendre sur Moodle avec votre fichier `tp4.ipynb`.